In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

Tensorflow is not available


c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val= split_data2(data, 0.2)
data_train_val = data_train + data_val

In [3]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit()

[I 2024-01-02 22:55:23,901] A new study created in memory with name: P3Beta


In [4]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [5]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe

,ItemID
UserID,
0,"[22146, 674, 15344, 587, 2843, 1449, 3413, 111..."
1,"[9728, 2548, 83, 89, 227, 841, 517, 2612, 812,..."
2,"[5777, 10028, 1354, 1427, 4502, 5136, 5375, 45..."
3,"[8073, 227, 812, 589, 2612, 44, 819, 841, 2162..."
4,"[20641, 12793, 19369, 4709, 17723, 18001, 8688..."
...,...
12633,"[2809, 2817, 2807, 2811, 2818, 2815, 7069, 396..."
12634,"[11498, 3949, 3955, 3943, 17082, 17072, 2810, ..."
12635,"[257, 472, 470, 259, 248, 3297, 17223, 2835, 1..."


In [6]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,22146
0,674
0,15344
0,587
0,2843
...,...
12637,19760
12637,16596
12637,19019


In [7]:
#import pandas as pd
#import numpy as np
#from sklearn.model_selection import KFold
#from scipy import sparse as sps
#
## Carica il tuo dataset
## data_train = ...
#
## Definisci il numero di fold desiderato
#k = 5
#
## Inizializza l'oggetto KFold
#kf = KFold(n_splits=k, shuffle=True, random_state=42)
#
#print(kf)
#
#folds = list(kf.split(training_dataframe))
#
#print(len(folds))
#
#cutoff = 50  # Scegli il valore desiderato per il cutoff
#
#for fold, (train_index, val_index) in enumerate(folds):
#    # Dividi il dataset in training e validation
#    train_fold, val_fold = data_val[train_index], data_val[val_index]
#
#    # Valuta il modello su val_fold
#    predictions = []
#
#    for user_id in range(val_fold.shape[0]):
#        recommendations = candidate_recommender.recommend(user_id, cutoff=cutoff, remove_seen_flag=True)
#        predictions.append(recommendations)
#
#
#    # Flatten predictions e val_fold per calcolare le etichette
#    predictions_flat = list(chain.from_iterable(predictions))
#    val_fold_coo = sps.coo_matrix(val_fold)
#    correct_recommendations = pd.DataFrame({"UserID": val_fold_coo.row, "ItemID": val_fold_coo.col})
#    
#    # Calcola le etichette sulla base della corrispondenza tra le previsioni e val_fold
#    labels = [tuple(item) in correct_recommendations.itertuples(index=False, name=None) for item in predictions_flat]
#
#    # Aggiungi le etichette al dataframe
#    training_dataframe.loc[val_index, f"Label_fold_{fold}"] = labels
#
## Aggiungi una colonna "Label" al dataframe che rappresenta l'OR logico delle etichette dei fold
#training_dataframe["Label"] = training_dataframe.filter(like="Label_fold").any(axis=1)
#
## Rimuovi le colonne temporanee dei fold
#training_dataframe.drop(training_dataframe.filter(like="Label_fold"), axis=1, inplace=True)

In [8]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})

correct_recommendations

,UserID,ItemID
0,0,0
1,0,3
2,0,9
3,0,16
4,0,17
...,...,...
95741,12637,10899
95742,12637,11823
95743,12637,13249
95744,12637,16594


In [9]:
#training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
#training_dataframe["Label"] = training_dataframe["Exist"] == "both"
#training_dataframe.drop(columns = ['Exist'], inplace=True)
#
#training_dataframe

In [10]:
for user_id in range(n_users):
    training_dataframe.loc[user_id, "Label"] = list(reversed(list(range(1, cutoff+1))))
training_dataframe

,ItemID,Label
UserID,,
0,22146,50.0
0,674,49.0
0,15344,48.0
0,587,47.0
0,2843,46.0
...,...,...
12637,19760,5.0
12637,16596,4.0
12637,19019,3.0


In [11]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit()

easer_study = optuna.create_study(
    study_name="Easer",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit()

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit()


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2024-01-02 22:55:41,873] A new study created in memory with name: P3Alpha


P3alphaRecommender: Similarity column 22222 (100.0%), 3066.97 column/sec. Elapsed time 7.25 sec


[I 2024-01-02 22:55:50,613] A new study created in memory with name: Easer


EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.72 min


[I 2024-01-02 23:02:14,450] A new study created in memory with name: ItemKNNCF


Similarity column 22222 (100.0%), 6834.53 column/sec. Elapsed time 3.25 sec


In [12]:
import scipy.sparse as sps

training_dataframe
training_dataframe = training_dataframe.set_index('UserID')

for user_id in range(n_users):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

training_dataframe

KeyError: "None of ['UserID'] are in the columns"

In [ ]:
#item_popularity = np.ediff1d(sps.csc_matrix(data_train_val).indptr)
#training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
#user_popularity = np.ediff1d(sps.csr_matrix(data_train_val).indptr)
#training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
#training_dataframe

KeyError: 'UserID'

In [ ]:
training_dataframe = training_dataframe.set_index('ItemID')
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [ ]:
training_dataframe
#groups = training_dataframe.groupby("UserID").size().values
#train_recs = training_dataframe.drop(columns=["Label"])
#train_labs = training_dataframe["Label"]

,ItemID,Label,item_popularity
0,22146,50.0,1
1,674,49.0,426
2,15344,48.0,6
3,587,47.0,713
4,2843,46.0,7
...,...,...,...
631895,19760,5.0,6
631896,16596,4.0,2
631897,19019,3.0,4
631898,1478,2.0,371


In [ ]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-02 22:49:50,722] A new study created in memory with name: XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)


In [ ]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_labs,
        train_recs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

#Trial 6 finished with value: 0.12485196495596997 and parameters: {'objective': 'rank:ndcg', 'n_estimators': 336, 'learning_rate': 0.023887567768263025, 'reg_alpha': 0.01495634916569455, 'reg_lambda': 0.004873292863879341, 'max_depth': 4, 'max_leaves': 3, 'grow_policy': 'depthwise', 'booster': 'gbtree'}. Best is trial 6 with value: 0.12485196495596997.

[W 2024-01-02 22:49:55,095] Trial 0 failed with parameters: {'objective': 'rank:pairwise', 'n_estimators': 483, 'learning_rate': 0.016915890430890663, 'reg_alpha': 0.003977270913995904, 'reg_lambda': 0.0004372708105376502, 'max_depth': 8, 'max_leaves': 3, 'grow_policy': 'lossguide', 'booster': 'dart'} because of the following error: NameError("name 'groups' is not defined").
Traceback (most recent call last):
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Francesco1\AppData\Local\Temp\ipykernel_2908\513521712.py", line 31, in objective
    group=groups,
NameError: name 'groups' is not defined
[W 2024-01-02 22:49:55,097] Trial 0 failed with value None.


NameError: name 'groups' is not defined

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
)
XGB_model.fit(
    training_dataframe.drop(columns=["Label"]),
    training_dataframe["Label"],
    group=groups,
    verbose=False
)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)